In [56]:
from py2neo import Graph
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
neo4j_password = os.getenv('NEO4J_PASSWORD')

In [57]:
graph = Graph("neo4j://localhost:7687", auth=("neo4j", neo4j_password))

In [58]:
def get_connected_nodes(node_name):
    # Define the Cypher query to find connected nodes and return both Name and Desc properties
    query = """
    MATCH (n {Name: $node_name})-[:PARENT_OF]-(connected_nodes)
    RETURN connected_nodes.Name AS name, connected_nodes.Desc AS desc
    """
    
    # Execute the query
    results = graph.run(query, node_name=node_name)
    
    # Extract the Name and Desc of each connected node
    connected_nodes_info = []
    for record in results:
        name = record["name"]
        desc = record["desc"]
        connected_nodes_info.append({"Name": name, "Desc": desc})
    
    return connected_nodes_info

In [69]:
from dotenv import load_dotenv
import os 
load_dotenv('.env')

True

In [70]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate


In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(name="current_output", description="Output from the current agent that will be passed to the next agent. If there is no next agent, put None"),
    ResponseSchema(name="next_agent", description="The next agent to handle the request."),
    ResponseSchema(name="next_task", description="The task or action for the next agent."),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)


In [ ]:
format_instructions = output_parser.get_format_instructions()

In [75]:
search_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a highly skilled assistant designed to search and answer user's general queries
            Given a list of connected agents with their names and description, if required return the name of the next agent you require for your assistance as well as give an instruction for it.
               List of agents : {Connected_agents}\n
               Format Instructions : {format_instructions}"""

        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

greeting_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a friendly AI that greets people warmly when they start a conversation and says goodbye appropriately when they mention leaving. 

                If someone says something about going away, respond with a proper goodbye.
                If they just greet you or talk normally, respond with a friendly greeting.
                Given a list of connected agents with their names and description, if required return the name of the next agent you require for your assistance as well as give an instruction for it.
               List of agents : {Connected_agents} \n
               Format Instructions : {format_instructions}"""
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


math_prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a highly skilled assistant with expertise in solving math problems. Ensure accurate calculations and provide clear explanations for mathematical concepts. Use the tavily_search_results_json tool for additional information if needed.
                Given a list of connected agents with their names and description, if required return the name of the next agent you require for your assistance as well as give an instruction for it.
               List of agents : {Connected_agents} \n
               Format Instructions : {format_instructions}"""
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


tool_prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an AI assistant with access to powerful tools for calculations. Use these calculation tools when you need to perform precise calculations. 
               Given a list of connected agents with their names and description, if required return the name of the next agent you require for your assistance as well as give an instruction for it.
               List of agents : {Connected_agents} \n
               Format Instructions : {format_instructions}"""
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

supervisor_prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an AI assistant who is the first agent tasked to find the appropriate next agent given the task. 
               Given a list of connected agents with their names and description, if required return the name of the next agent you require for your assistance as well as give an instruction for it.
               List of agents : {Connected_agents} \n
               Format Instructions : {format_instructions}"""
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [76]:
import os
import openai

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')


In [62]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, openai_api_key=OPENAI_API_KEY)

In [64]:
search = TavilySearchResults(tavily_api_key = TAVILY_API_KEY)
tools = [search]

In [65]:
from langchain.agents import create_tool_calling_agent

agent2 = create_tool_calling_agent(llm, tools, search_prompt)

In [ ]:
supervisor = create_tool_calling_agent(llm, tools, supervisor_prompt)

In [67]:
agent21 = create_tool_calling_agent(llm, tools=tools, prompt= greeting_prompt)

In [ ]:
# Agent 1: Generally good at math questions
agent1 = create_tool_calling_agent(
    llm=llm,
    tools=[search],
    prompt=math_prompt,
)

In [ ]:
agent11 = create_tool_calling_agent(
    llm=llm,
    tools=[search],
    prompt=tool_prompt,
)

In [ ]:
from langchain.prompts import PromptTemplate

choosing_template = """
You are an agent with multiple other AI agents for your assistance.

AI: 
"""

# Initialize the prompt template
choosing_prompt = PromptTemplate(input_variables=["Connected_agents"], template=choosing_template, partial_variables={"format_instructions": format_instructions})